# VGG16 with DataAugmentation 

In [1]:
# Vgg16
#RGB Images
# 128X128 resized images 

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model

Using TensorFlow backend.


###### Loading data
Intially the datas are structured to input to the convolutional layer by converting the images in to numpy array, normalized (changing the range of pixel intensity values) and reshaped accordingly. Finally, stored in the path shown below.

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/train/x_data_all.npy')
y_train = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (17194, 128, 128, 3) y_train.shape: (17194, 9)
x_valid.shape: (2558, 128, 128, 3) y_valid.shape: (2558, 9)


In [4]:
batch_size = 24   # The batch size is set to 32 which is different with RGB_224 
resize = 128     # The image is reduced to 128x128 which is different with RGB_224
n_class = 9

## VGG16 Model 

In [5]:
from keras.applications.vgg16 import VGG16, preprocess_input


inputs = Input (shape = (resize, resize, 3))


model_vgg16_conv = VGG16(weights='imagenet', include_top=False, input_shape = (224,224,3))
model_vgg16_conv.summary()

W0522 01:44:14.220958 140713366554368 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:

output_vgg16 = model_vgg16_conv(inputs)



x=GlobalAveragePooling2D()(output_vgg16)

dense_layer = Dense(1024, activation = 'relu')(x)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(1024, activation = 'relu')(dense_layer)
dense_layer = BatchNormalization()(dense_layer)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(512, activation = 'relu')(dense_layer)

final_output = Dense(n_class, activation = 'softmax')(dense_layer)


final_model = Model ([inputs], final_output)

final_model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              525312    
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096

## Model Saving and compling 

In [7]:

save_name = 'Comp_VGG16_RGB_128_Ag'   
save_path = './checkpoints/' + save_name   # saved path 

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

final_model = multi_gpu_model(final_model, gpus=2)


sgd = optimizers.SGD(lr = 0.001)  

final_model.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])


    
final_model.fit(x_train, y_train, 
                     batch_size=batch_size,
                    validation_data=(x_valid, y_valid),       
                    epochs=60,
                    verbose=1,
                    callbacks=callback_list)

final_model.save(save_path + '/epoch-finish.h5')
final_model.save_weights(save_path +'/epoch-finish_weights.h5')

W0522 01:44:18.486603 140713366554368 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0522 01:44:20.255704 140713366554368 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0522 01:44:20.263372 140713366554368 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0522 01:44:20.264061 140713366554368 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Train on 17194 samples, validate on 2558 samples
Epoch 1/60
17194/17194 [==============================] - 54s 3ms/step - loss: 2.6452 - accuracy: 0.1524 - val_loss: 1.9397 - val_accuracy: 0.3049

Epoch 00001: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0001.h5


W0522 01:45:26.993912 140713366554368 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/60
17194/17194 [==============================] - 48s 3ms/step - loss: 2.0655 - accuracy: 0.2832 - val_loss: 1.5925 - val_accuracy: 0.4433

Epoch 00002: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0002.h5
Epoch 3/60
17194/17194 [==============================] - 48s 3ms/step - loss: 1.6514 - accuracy: 0.4201 - val_loss: 1.3540 - val_accuracy: 0.6388

Epoch 00003: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0003.h5
Epoch 4/60
17194/17194 [==============================] - 48s 3ms/step - loss: 1.3154 - accuracy: 0.5511 - val_loss: 1.1107 - val_accuracy: 0.7220

Epoch 00004: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0004.h5
Epoch 5/60
17194/17194 [==============================] - 48s 3ms/step - loss: 1.0667 - accuracy: 0.6502 - val_loss: 0.9940 - val_accuracy: 0.7275

Epoch 00005: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0005.h5
Epoch 6/60
17194/17194 [==============================] - 48s 3ms/step - loss: 0.850

Epoch 38/60
17194/17194 [==============================] - 48s 3ms/step - loss: 0.0580 - accuracy: 0.9881 - val_loss: 0.6240 - val_accuracy: 0.8104

Epoch 00038: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0038.h5
Epoch 39/60
17194/17194 [==============================] - 48s 3ms/step - loss: 0.0508 - accuracy: 0.9912 - val_loss: 0.7062 - val_accuracy: 0.7830

Epoch 00039: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0039.h5
Epoch 40/60
17194/17194 [==============================] - 48s 3ms/step - loss: 0.0511 - accuracy: 0.9903 - val_loss: 0.6872 - val_accuracy: 0.7869

Epoch 00040: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0040.h5
Epoch 41/60
17194/17194 [==============================] - 48s 3ms/step - loss: 0.0525 - accuracy: 0.9891 - val_loss: 0.7297 - val_accuracy: 0.7920

Epoch 00041: saving model to ./checkpoints/Comp_VGG16_RGB_128/checkpoint-0041.h5
Epoch 42/60
17194/17194 [==============================] - 48s 3ms/step - loss: 